In [3]:
import nglview as nv
import pytraj as pt 


print(dir(nv))


traj = pt.load(b, top=a); # "C499thistest_PDB.pdb"
nv.show_pytraj(traj)


['ASEStructure', 'ASETrajectory', 'BACKENDS', 'BiopythonStructure', 'DatasourceRegistry', 'FileStructure', 'HTMDTrajectory', 'IODataStructure', 'IOTBXStructure', 'MDAnalysisTrajectory', 'MDTrajTrajectory', 'NGLWidget', 'OpenbabelStructure', 'ParmEdTrajectory', 'PdbIdStructure', 'ProdyStructure', 'ProdyTrajectory', 'Psi4Structure', 'PyTrajTrajectory', 'QCElementalStructure', 'RdkitStructure', 'RosettaStructure', 'SchrodingerStructure', 'SchrodingerTrajectory', 'SimpletrajTrajectory', 'Structure', 'TextStructure', 'Trajectory', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_frontend', '_jupyter_nbextension_paths', '_version', 'adaptor', 'base', 'base_adaptor', 'color', 'component', 'config', 'data_source', 'datafiles', 'default', 'demo', 'interpolate', 'js_utils', 'layout', 'parameters', 'player', 'register_backend', 'remote_thread', 'representation', 'shape', 'show', 'show_ase', 'show_aset

NGLWidget(max_frame=5)

In [4]:
import nglview as nv 
import pytraj as pt
def rand_color():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())
def nglmask(traj, mask):
  return "@"+",".join([str(i) for i in traj.top.select(mask)])
def show_complex(traj, viewer_dic):
  viewer = nv.show_pytraj(traj)
  viewer[0].clear_representations()
  for i in viewer_dic:
    mask = nglmask(traj, i)
    viewer[0].add_representation(viewer_dic[i], selection=mask)
  viewer.center(selection=mask)
  viewer.stage.set_parameters(backgroundColor="#f9fcfd")
  return viewer



stride   = 1
masks    = {
  ":LIG,MDL<:15&!:SOL,T3P":"ribbon", 
  ":LIG,MDL<:5&!:SOL,T3P,WAT":"line", 
  ":LIG,MDL":"ball+stick", 
}; 
trajfile = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_TRJ.nc"; 
topfile  = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_PDB.pdb"; 
traj = pt.load(trajfile, top=topfile, stride=stride)
traj.top.set_reference(traj[0])
viewer = show_complex(traj, masks)
# print(dir(viewer))
viewer

NGLWidget(max_frame=5)

In [2]:
from BetaPose import utils, view
import nglview as nv 
import pytraj as pt


STRIDE   = 1
REPR    = {
  ":LIG,MDL<:15&!:SOL,T3P":"ribbon", 
  ":LIG,MDL<:5&!:SOL,T3P,WAT":"line", 
  ":LIG,MDL":"ball+stick", 
}; 

trajfile = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_TRJ.nc"; 
topfile  = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_PDB.pdb"; 

traj = pt.load(trajfile, top=topfile, stride=STRIDE);
traj.top.set_reference(traj[0]); 
viewer = view.show_complex(traj, REPR); 
dist, info = utils.PairwiseDistance(traj, ":LIG&!@H=", ":LIG<:6&@CA,C,N,O,CB")

print(dir(utils))
print(dist)
print(info)


# def add_distance(partner1, partner2):
#   for i, j in zip(partner1, partner2):
#     viewer[0].add_representation("distance", atomPair=[[f"@{i}", f"@{j}"]], 
#                                  color="blue", label_color="black", label_fontsize=0); 
#   return viewer

viewer = view.add_distance(viewer, info["gp1_ids"], info["gp2_ids"])
viewer

Calculating pairwise distance
['ASALig', 'ASALigOnly', 'DistanceLigPro', 'DrawGridMols', 'EmbeddingFactor', 'GetProteinByTraj', 'GetProteinMask', 'LIG_nha', 'PRO_nha', 'PRO_nhydrogen', 'PairwiseDistance', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cgenff2dic', 'cgenff2xml', 'cgenff2xmls', 'combineMOL2PDB', 'conflictfactor', 'distance_matrix', 'fetch', 'getAxisIndex', 'getPdbSeq', 'getPdbTitle', 'np', 'pt', 'smartsSupplier', 'tempfile']
[[4.604499  5.6068707 4.180211  4.980022  5.6989636 5.366957 ]
 [3.418143  4.471073  3.2104502 3.8788402 4.5949306 4.330844 ]
 [3.650944  4.3500557 3.6341288 3.9140913 3.9548807 4.1739964]
 ...
 [5.4082866 4.7461233 4.956728  4.623363  5.2951446 4.6468253]
 [4.3422337 5.504799  4.009166  4.410995  4.831408  4.832969 ]
 [4.99921   4.2226605 4.138965  3.5220633 5.017911  3.8611515]]
{'gp1_names': ['C1', 'N9', 'C7', 'C6', 'C5', 'C4', 'N8', 'C3', 'O10', 'C2'], 'gp1_ids': [64348, 64349, 64350, 64

NGLWidget(max_frame=5)

In [10]:
from BetaPose import utils, view
import nglview as nv 
import pytraj as pt
def nglmask(traj, mask):
  return "@"+",".join([str(i) for i in traj.top.select(mask)])
class TrajectoryViewer: 
  def __init__(self, traj):
    self.traj = traj; 
    self.viewer = nv.show_pytraj(traj); 
    
  def show_complex(self, viewer_dic):
    self.viewer[0].clear_representations(); 
    for rep in viewer_dic:
      mask = nglmask(self.traj, rep); 
      self.viewer[0].add_representation(viewer_dic[rep], selection=mask); 
    self.viewer.center(selection = mask); 
    self.viewer.stage.set_parameters(backgroundColor="#f9fcfd"); 
    return self.viewer
  def add_distance(self, partner1, partner2):
    for i, j in zip(partner1, partner2):
      self.viewer[0].add_representation("distance", atomPair=[[f"@{i}", f"@{j}"]],
                                   color="blue", label_color="black", label_fontsize=0.1);
    return self.viewer

  
STRIDE   = 1
REPR    = {
  ":LIG,MDL<:15&!:SOL,T3P":"ribbon", 
  ":LIG,MDL<:5&!:SOL,T3P,WAT":"line", 
  ":LIG,MDL":"ball+stick", 
}; 
trajfile = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_TRJ.nc"; 
topfile  = "/storage006/yzhang/clean_acgui/TMP_FOLDERS/thistest/singleMD/C499thistest/C499thistest_PDB.pdb"; 

traj = pt.load(trajfile, top=topfile, stride=STRIDE);
traj.top.set_reference(traj[0]); 
tv = TrajectoryViewer(traj); 
tv.show_complex(REPR); 
dist, info = utils.PairwiseDistance(traj, ":LIG&!@H=", ":LIG<:6&@CA,C,N,O,CB")
tv.add_distance(info["gp1_ids"], info["gp2_ids"])
tv.viewer

Calculating pairwise distance


NGLWidget(max_frame=5)